In [13]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [14]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if DCBS[lm[0]].__contains__(code):
                if low_prc(code, dt, lm, fcd[1]):
                    satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DCBS[dt][code]['dp']):
                lm = calculate_time(dt, 'p', 20)
                if DCBS[lm[0]].__contains__(code):
                    prc = get_prcs(code, lm)
                    lp = absolute_low(prc, 0)
                    rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]

def get_prcs(code, dts):
    prc = []
    
    for dt in dts:
        prc.append(DCBS[dt][code]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DCBS[dt][code]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = list(DCBS.keys())
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]



In [15]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
td
DCBS.keys()


dict_keys(['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-04', '2023-07-03'])

In [16]:
select = list(DCBS[td].keys())

# flt_cd = [['dp', 100, 130]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]









# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]


# 
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

# flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]




flt_cd = [['dp',100,130],['ytm', 0.5, 999], ['bl', 0, 7], ['trt',2,10]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]    
flt_cd = [['dp',100,130],['ytm', -1, 999], ['bl', 0, 7], ['trt',2,10]]

flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
for fcd in flt_cd:
    select = select_condition(select, td, fcd)

print(len(select))
    












rk_cd = [['dl', 30, 'a']]

rk_cd = [['lp', 30, 'a']]

rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 30, 'a']]
codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(codes)

5


5

In [17]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '货币资金',  round(DTBS['F'][DTBS['B'][r]['sc']]['2023-03']['hb'],2), '双低', DCBS[td][r]['dl'])

123014.SZ 凯发转债 300407.SZ 凯发电气 收盘价： 118.16 转股溢价率： 3.16 到期收益率： -163.35 换手率： 13.21 市值 1.151 货币资金 6.75 双低 121.32
123002.SZ 国祯转债 300388.SZ 节能国祯 收盘价： 108.362 转股溢价率： 30.75 到期收益率： -5.56 换手率： 1.8 市值 2.521 货币资金 12.77 双低 139.112
128021.SZ 兄弟转债 002562.SZ 兄弟科技 收盘价： 110.872 转股溢价率： 31.85 到期收益率： -10.91 换手率： 1.51 市值 2.651 货币资金 3.97 双低 142.722
123152.SZ 润禾转债 300727.SZ 润禾材料 收盘价： 126.9 转股溢价率： 40.89 到期收益率： -0.39 换手率： 2.28 市值 2.92 货币资金 1.28 双低 167.79000000000002
118007.SH 山石转债 688030.SH 山石网科 收盘价： 126.976 转股溢价率： 45.9 到期收益率： -1.18 换手率： 5.64 市值 2.673 货币资金 3.85 双低 172.876
